In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_colleges = pd.read_csv("../csv_clean/colleges.csv")
df_lycees = pd.read_csv("../csv_clean/lycees.csv")
df_transport = pd.read_csv("../csv_clean/transports.csv")
pd.set_option('display.precision', 15)

df_transport


In [ ]:
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display

# --- Liste des codes postaux disponibles ---
codes_postaux = sorted(set(df_colleges["code_postal"]).union(df_lycees["code_postal"]))
options_cp = ["Tous"] + [str(cp) for cp in codes_postaux]

# --- Widgets interactifs ---
dropdown_cp = widgets.Dropdown(
    options=options_cp,
    description="Code postal :",
    value="Tous"
)

checkbox_colleges = widgets.Checkbox(
    value=True,
    description="Collèges (rouge)"
)

checkbox_lycees = widgets.Checkbox(
    value=True,
    description="Lycées (bleu)"
)

checkbox_transport = widgets.Checkbox(
    value=False,
    description="Transports (vert)"
)

# --- Fonction pour mettre à jour la carte ---
def afficher_carte(code_postal, afficher_colleges, afficher_lycees, afficher_transports):
    # Filtrage des établissements selon code postal
    if code_postal == "Tous":
        cols = df_colleges
        lycees = df_lycees
    else:
        code_postal = int(code_postal)
        cols = df_colleges[df_colleges["code_postal"] == code_postal]
        lycees = df_lycees[df_lycees["code_postal"] == code_postal]
        transports = df_transport[df_transport["code_postal"] == code_postal] if "code_postal" in df_transport.columns else df_transport
    
    # Préparation des datasets à afficher
    etablissements = []
    if afficher_colleges:
        etablissements.append(("red", "Collège", cols))
    if afficher_lycees:
        etablissements.append(("blue", "Lycée", lycees))
    
    # Rien de coché pour les établissements ?
    if not etablissements and not afficher_transports:
        display(widgets.HTML("<b style='color:red;'>Coche au moins un type d'établissement ou transport.</b>"))
        return
    
    # Calcul du centre de la carte
    all_points = pd.concat([df for _, _, df in etablissements if not df.empty])
    if afficher_transports and not df_transport.empty:
        all_points = pd.concat([all_points, df_transport]) if not all_points.empty else df_transport
    if len(all_points) > 0:
        center_lat = all_points["lat"].mean()
        center_lon = all_points["lon"].mean()
    else:
        center_lat, center_lon = 45.75, 4.85  # coordonnées par défaut (Lyon)
    
    # Création de la carte
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")
    
    # Ajout des marqueurs
    for color, label, df in etablissements:
        for _, row in df.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.8,
                popup=f"{'🏫' if label=='Collège' else '🎓'} {row['nom']} ({label})<br>Code postal : {row['code_postal']}"
            ).add_to(m)
    
    # Ajout des transports si coché
    if afficher_transports:
        for _, row in df_transport.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color="green",
                fill=True,
                fill_color="green",
                fill_opacity=0.8,
                popup=row.get("nom", "Transport")
            ).add_to(m)
    
    display(m)

# --- Lier les widgets à la fonction via interactive_output ---
out = widgets.interactive_output(
    afficher_carte,
    {
        "code_postal": dropdown_cp,
        "afficher_colleges": checkbox_colleges,
        "afficher_lycees": checkbox_lycees,
        "afficher_transports": checkbox_transport
    }
)

# --- Affichage de l'interface (widgets + zone d'affichage) ---
ui = widgets.VBox([dropdown_cp, widgets.HBox([checkbox_colleges, checkbox_lycees, checkbox_transport])])
display(ui, out)